In [1]:
import torch
from torch import nn
from torch.nn import BCEWithLogitsLoss
import torchsummary as summary

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, XLNetTokenizer, XLNetModel, XLNetLMHeadModel, XLNetConfig

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm, trange

C:\Users\hp\miniconda3\envs\tf_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.empty_cache()

In [3]:
'''
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/')


df=pd.read_csv('BERT_data.csv')
df=df[~(df['content']=='nan')]
df['content']=df['content'].astype(str)
df['subject']=df['subject'].astype(str)
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n\nimport os\nos.chdir('/content/drive/My Drive/')\n\n\ndf=pd.read_csv('BERT_data.csv')\ndf=df[~(df['content']=='nan')]\ndf['content']=df['content'].astype(str)\ndf['subject']=df['subject'].astype(str)\n"

In [4]:
df=pd.read_csv('BERT_data.csv')
df['content']=df['content'].astype(str)
df['subject']=df['subject'].astype(str)

In [5]:
df['cat_1_level_2']=df['cat_1_level_2']-1
df['cat_1_level_2'].unique()

array([0, 2, 3, 4, 1, 6, 5, 7], dtype=int64)

In [6]:
df=df[~(df['cat_1_level_2']==6)]
df=df[~(df['cat_1_level_2']==7)]
df['cat_1_level_2'].unique()

array([0, 2, 3, 4, 1, 5], dtype=int64)

In [7]:
# to count max word length
def word_count(text):
    words=text.split()
    return len(words)
max(df['content'].apply(word_count))

2951

In [8]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

class Dataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.labels=df['cat_1_level_2']
        self.text=[tokenizer(text,padding='max_length',max_length=512,truncation=True,return_tensors="pt") for text in df['content']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.text[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [9]:
df

,message_id,subject,content,cat_1_level_2
0,<9831685.1075855725804.JavaMail.evans@thyme>,re confidenti employe informationlenhart,also need know base salari jay reitmey moniqu ...,0
1,<21041312.1075855725847.JavaMail.evans@thyme>,re person and confidenti compens inform,thank inform would help would send detail work...,2
2,<21261996.1075858638025.JavaMail.evans@thyme>,re western wholesal activ gas power conf call ...,guy coordin make sure someon listen confer cal...,3
3,<20399547.1075857614321.JavaMail.evans@thyme>,re confidenti employe informationdutch quigley,thx jeani slone pm john cc confidenti employe ...,4
4,<860767.1075849626951.JavaMail.evans@thyme>,re hp confidenti intern document,excel updat thank put togeth dalepatrick let r...,0
...,...,...,...,...
1485,<18361957.1075861368310.JavaMail.evans@thyme>,confidenti due dilig info g,lot confidenti inform item detail month pl cov...,0
1486,<29049822.1075842045435.JavaMail.evans@thyme>,california dim bulb,may seen first came back worth rerun,0
1487,<205897.1075861997314.JavaMail.evans@thyme>,re confidenti ee info,mark email fax ani written document give heidi...,4
1488,<23743848.1075863311776.JavaMail.evans@thyme>,confidenti inform and secur trade,william william enron wholesal servic offic ch...,0


In [10]:
X_train, X_test, y_train, y_test = train_test_split(df[['message_id','subject','content']], df['cat_1_level_2'], test_size=0.4, stratify=df['cat_1_level_2'])

In [11]:
X_train['cat_1_level_2']=y_train
X_test['cat_1_level_2']=y_test
df_train=X_train
df_val=X_test
df_test=0

In [12]:
df_train.reset_index(inplace=True)
df_train=df_train.drop('index',axis=1)

df_val.reset_index(inplace=True)
df_val=df_val.drop('index',axis=1)
'''
df_test.reset_index(inplace=True)
df_test=df_test.drop('index',axis=1)
'''
print(len(df_train),len(df_val))

874 584


In [13]:
class XLNetClassifier(nn.Module):
    def __init__(self, dropout=0.3):
        super(XLNetClassifier, self).__init__() 
        self.xlnet = XLNetModel.from_pretrained('xlnet-base-cased', return_dict=True)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(self.xlnet.config.hidden_size, 6)
        torch.nn.init.kaiming_uniform_(self.linear.weight, nonlinearity='relu')
        self.relu = nn.ReLU()
        
    def forward(self, input_ids, attention_mask):
        
        outputs = self.xlnet(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # Get the CLS token representation
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer


In [14]:
'''
class XLNetClassifier(nn.Module):

    def __init__(self, dropout=0.3):

        super(XLNetClassifier, self).__init__()
        self.xlnet = XLNetModel.from_pretrained('xlnet-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 6)
        torch.nn.init.kaiming_uniform_(self.linear.weight, nonlinearity='relu')
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _,pooled_output = self.xlnet(input_ids= input_id, attention_mask=mask)
        dropout_output = self.dropout(pooled_output[0])
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer
'''

"\nclass XLNetClassifier(nn.Module):\n\n    def __init__(self, dropout=0.3):\n\n        super(XLNetClassifier, self).__init__()\n        self.xlnet = XLNetModel.from_pretrained('xlnet-base-cased')\n        self.dropout = nn.Dropout(dropout)\n        self.linear = nn.Linear(768, 6)\n        torch.nn.init.kaiming_uniform_(self.linear.weight, nonlinearity='relu')\n        self.relu = nn.ReLU()\n\n    def forward(self, input_id, mask):\n\n        _,pooled_output = self.xlnet(input_ids= input_id, attention_mask=mask)\n        dropout_output = self.dropout(pooled_output[0])\n        linear_output = self.linear(dropout_output)\n        final_layer = self.relu(linear_output)\n\n        return final_layer\n"

In [15]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [16]:
EPOCHS = 300
model = XLNetClassifier()
LR = 0.000001

In [17]:
n=0
for x in model.state_dict():
    n=n+1
    print(x)
n

xlnet.mask_emb
xlnet.word_embedding.weight
xlnet.layer.0.rel_attn.q
xlnet.layer.0.rel_attn.k
xlnet.layer.0.rel_attn.v
xlnet.layer.0.rel_attn.o
xlnet.layer.0.rel_attn.r
xlnet.layer.0.rel_attn.r_r_bias
xlnet.layer.0.rel_attn.r_s_bias
xlnet.layer.0.rel_attn.r_w_bias
xlnet.layer.0.rel_attn.seg_embed
xlnet.layer.0.rel_attn.layer_norm.weight
xlnet.layer.0.rel_attn.layer_norm.bias
xlnet.layer.0.ff.layer_norm.weight
xlnet.layer.0.ff.layer_norm.bias
xlnet.layer.0.ff.layer_1.weight
xlnet.layer.0.ff.layer_1.bias
xlnet.layer.0.ff.layer_2.weight
xlnet.layer.0.ff.layer_2.bias
xlnet.layer.1.rel_attn.q
xlnet.layer.1.rel_attn.k
xlnet.layer.1.rel_attn.v
xlnet.layer.1.rel_attn.o
xlnet.layer.1.rel_attn.r
xlnet.layer.1.rel_attn.r_r_bias
xlnet.layer.1.rel_attn.r_s_bias
xlnet.layer.1.rel_attn.r_w_bias
xlnet.layer.1.rel_attn.seg_embed
xlnet.layer.1.rel_attn.layer_norm.weight
xlnet.layer.1.rel_attn.layer_norm.bias
xlnet.layer.1.ff.layer_norm.weight
xlnet.layer.1.ff.layer_norm.bias
xlnet.layer.1.ff.layer_1.weig

208

In [18]:
# freeze first 7 layers 
n=0
for param in model.parameters():
    n=n+1
    param.requires_grad = False
    if n==(201-68):
        break

In [19]:
'''
train, val = Dataset(df_train), Dataset(df_val)
train_dataloader = torch.utils.data.DataLoader(train, batch_size=20)
val_dataloader = torch.utils.data.DataLoader(val, batch_size=20)

'''

'\ntrain, val = Dataset(df_train), Dataset(df_val)\ntrain_dataloader = torch.utils.data.DataLoader(train, batch_size=20)\nval_dataloader = torch.utils.data.DataLoader(val, batch_size=20)\n\n'

In [20]:
# train[321][0]['input_ids']

In [21]:
'''
train, val = Dataset(df_train), Dataset(df_val)

model = model.cuda()
train_dataloader = torch.utils.data.DataLoader(train, batch_size=2)
for train_input, train_label in tqdm(train_dataloader):
                train_label = train_label.to(device)
                mask = train_input['attention_mask'].squeeze(1).to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
                output = model(input_id, mask)
                print(mask)
'''

"\ntrain, val = Dataset(df_train), Dataset(df_val)\n\nmodel = model.cuda()\ntrain_dataloader = torch.utils.data.DataLoader(train, batch_size=2)\nfor train_input, train_label in tqdm(train_dataloader):\n                train_label = train_label.to(device)\n                mask = train_input['attention_mask'].squeeze(1).to(device)\n                input_id = train_input['input_ids'].squeeze(1).to(device)\n                output = model(input_id, mask)\n                print(mask)\n"

In [22]:
# input_id.shape

NameError: name 'input_id' is not defined

In [23]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=10)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=10)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0
            n=0
            for train_input, train_label in tqdm(train_dataloader):
                train_label = train_label.to(device)
                mask = train_input['attention_mask'].squeeze(1).to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)


                output = model(input_id, mask)
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()

                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()

            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].squeeze(1).to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')



train(model, df_train, df_val, LR, EPOCHS)

100%|██████████████████████████████████████████████████████████████████████████████████| 88/88 [01:19<00:00,  1.11it/s]


Epochs: 1 | Train Loss:  0.248                 | Train Accuracy:  0.319                 | Val Loss:  0.195                 | Val Accuracy:  0.401


100%|██████████████████████████████████████████████████████████████████████████████████| 88/88 [01:17<00:00,  1.14it/s]


Epochs: 2 | Train Loss:  0.188                 | Train Accuracy:  0.429                 | Val Loss:  0.172                 | Val Accuracy:  0.437


100%|██████████████████████████████████████████████████████████████████████████████████| 88/88 [01:17<00:00,  1.14it/s]


...

Epochs: 244 | Train Loss:  0.022                 | Train Accuracy:  0.905                 | Val Loss:  0.299                 | Val Accuracy:  0.515


100%|██████████████████████████████████████████████████████████████████████████████████| 88/88 [01:16<00:00,  1.15it/s]


Epochs: 245 | Train Loss:  0.021                 | Train Accuracy:  0.911                 | Val Loss:  0.299                 | Val Accuracy:  0.497


 48%|███████████████████████████████████████▏                                          | 42/88 [00:37<00:41,  1.12it/s]


KeyboardInterrupt: 

In [ ]:
'''
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

evaluate(model, df_test)
'''

In [24]:
torch.save(model.state_dict(), 'XLNet_model_freeze.pth')